<a href="https://colab.research.google.com/github/custom-hyper/Research/blob/main/watchlist_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Connect to FNS { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pipreqs
!pip install configparser
!pip install psycopg2
!pip install dash

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=7809bf19a5970a65be466eb2e9e7a88e6818ac0ad2554ddae9cad28a363aa5cc
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends
     |████████████████████████████████| 9.6 MB 7.6 MB/s 
     |████████████████████████████████| 357 kB 55.5 MB/s 


In [ ]:
#@title Import Project  Dependencies { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm
import os
import sqlite3 as sq
import psycopg2 as pg
import configparser
from sqlalchemy import create_engine
from pycoingecko import CoinGeckoAPI
import glob
from pathlib import Path
from __future__ import division

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Setup SQLite


In [ ]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

In [ ]:
#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('deFi_lama_total_TVL',),
 ('proprietary_Twitter_Partnership_ohlc',),
 ('proprietary_master_id',),
 ('CoinGecko_ohlc',),
 ('proprietary_tweets_extract',),
 ('proprietary_Twitter_partnerships_announcements',),
 ('deFi_lama_tvl',),
 ('proprietary_airtable_watch_all',),
 ('proprietary_Twitter_id',),
 ('deFi_lama_general',),
 ('proprietary_dashboard',),
 ('watchlist_CoinGecko_ohlc',)]

[('deFi_lama_total_TVL',),
 ('proprietary_Twitter_Partnership_ohlc',),
 ('proprietary_master_id',),
 ('CoinGecko_ohlc',),
 ('proprietary_tweets_extract',),
 ('proprietary_Twitter_partnerships_announcements',),
 ('deFi_lama_tvl',),
 ('proprietary_airtable_watch_all',),
 ('proprietary_Twitter_id',),
 ('deFi_lama_general',),
 ('proprietary_dashboard',),
 ('watchlist_CoinGecko_ohlc',)]

In [ ]:
#@title Load Token ID to SQLite { display-mode: "code" }
proprietary_airtable_watch_all  = "proprietary_airtable_watch_all"


df_proprietary_airtable_shortlisted = pd.read_sql("""
          select *
          from {} 
        
          WHERE status = 'Shortlisted'
          And CoinGecko_id not null
          """.format(proprietary_airtable_watch_all), conn_SQLite)

df_proprietary_airtable_shortlisted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 61 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     22 non-null     object 
 1   close watch            17 non-null     object 
 2   CoinGecko_id           24 non-null     object 
 3   yahoo_id               0 non-null      object 
 4   update                 0 non-null      object 
 5   status                 24 non-null     object 
 6   Count (Memo)           24 non-null     int64  
 7   org_type               14 non-null     object 
 8   theme                  19 non-null     object 
 9   theme_rating           19 non-null     object 
 10  investors_list         11 non-null     object 
 11  Investors_found        2 non-null      object 
 12  thesis                 19 non-null     object 
 13  personal_questions     3 non-null      object 
 14  vision                 2 non-null      object 
 15  descript

In [ ]:

token_list = df_proprietary_airtable_shortlisted['CoinGecko_id']
frames = []

try:

  for i in tqdm(token_list[: 3]):

    #General Information table
    response = requests.get("https://api.coingecko.com/api/v3/coins/{}".format(i))
    output = response.text
    json_parsed = json.loads(output)
    
    df= pd.DataFrame()
    
    df['market_cap'] = json_parsed['market_data']['market_cap']['usd']
    df['circulating_supply'] = json_parsed['market_data']['circulating_supply']  
    df['current_price'] = json_parsed['market_data']['current_price']['usd']  
    df['fdv_to_tvl_ratio'] = json_parsed['market_data']['fdv_to_tvl_ratio']  
    #df['fully_diluted_valuation'] = json_parsed['market_data']['fully_diluted_valuation']['usd']
    df['price_change_24h'] = json_parsed['market_data']['price_change_24h'] 
    #df['max_supply'] = json_parsed['market_data']['max_supply']['usd']
    df['mcap_to_tvl_ratio'] = json_parsed['market_data']['mcap_to_tvl_ratio']
    df['total_supply'] = json_parsed['market_data']['total_supply']
    #df['total_value_locked'] = json_parsed['market_data']['total_value_locked']
    df['total_volume'] = json_parsed['market_data']['total_volume']['usd']

    df['dollar_volume_rotation']  = df['total_volume'] / df['market_cap']
    #df['supply_expansion_multiple'] = df['max_supply']/df['circulating_supply']
    
    df['CoinGecko_id'] = str(i)
    df['price_change_percentage_7d'] = json_parsed['market_data']['price_change_percentage_7d']
    df['price_change_percentage_14d'] = json_parsed['market_data']['price_change_percentage_14d']
    df['price_change_percentage_30d'] = json_parsed['market_data']['price_change_percentage_30d']
    df['price_change_percentage_60d'] = json_parsed['market_data']['price_change_percentage_60d']
    df['price_change_percentage_200d'] = json_parsed['market_data']['price_change_percentage_200d']
    df['price_change_percentage_1y'] = json_parsed['market_data']['price_change_percentage_1y']

    frames.append(df)
    time.sleep(60/50)
except ConnectionError:
    time.sleep(60)
single = pd.concat(frames) #Careful, this line must be outside of the for loop
single['prop_timestamp'] = int( time.time_ns() / 1000 )



"""    
    df['circulating_versus_max'] = circulating_supply/total_supply
    df['market_cap_dilution_ratio'] =  fully_diluted_valuation / market_cap
      
          
    df['dollar_volume_rotation'] = dollar_volume_rotation
    df['['supply_expansion_multiple'] = supply_expansion_multiple
    df['circulating_versus_max'] = circulating_versus_max
    df['market_cap_dilution_ratio'] = market_cap_dilution_ratio - 1
    except ZeroDivisionError:
      print('null')
      

    
    #df_market = pd.DataFrame(data=mkdata, columns = ['market_cap','current_price','fdv_to_tvl_ratio','fully_diluted_valuation','mcap_to_tvl_ratio'])
    
    print(df_market)


    
    print(dict_return)



    category = json_parsed['categories']

    print('')  
    print('CATEGORIES')  
    print(category)
    
    
    
    community_data = json_parsed['community_data']

    df_community_data = pd.DataFrame(data=community_data, index=[0])
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    print('')  
    print('COMMUNITY DATA')
    print(community_data)

    from google.colab import data_table
    data_table.DataTable(df_community_data)
    
    community_score = json_parsed['community_score']
    
    description  = [] 
    pd.set_option('display.width', 10)
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    description = json_parsed['description']['en']
    
    print('')  
    print('DESCRIPTION: ')
    print(description)

    print('')  
    print('DEVELOPER ACTIVITY')
    developer_data = json_parsed['developer_data']
    print(developer_data)
    
    genesis_date = json_parsed['genesis_date']
    print('')
    print('GENESIS DATE')
    print(genesis_date)
    
    links = json_parsed['links']
    
    pd.set_option('display.width', 10)

    print('')
    print('WEB Links')
    print(links)

    #Status update

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}/status_updates".format(i))
    output = response.text
    json_parsed_Updates = json.loads(output)
    print('')
    print('Recent updates: ')
    print(json_parsed_Updates)
    try:  
      a = cg.get_coin_ohlc_by_id(id=i, vs_currency='usd', days = 'max')
    except:
        pass
    df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])
    
    
    
    fig2 = go.Figure(data=[go.Candlestick(x=df['Date'],open=df['Open'], high=df['High'],low=df['Low'], close=df['Close'])])
    
    
    fig2.show()
"""   


  0%|          | 0/3 [00:00<?, ?it/s]

'    \n    df[\'circulating_versus_max\'] = circulating_supply/total_supply\n    df[\'market_cap_dilution_ratio\'] =  fully_diluted_valuation / market_cap\n      \n          \n    df[\'dollar_volume_rotation\'] = dollar_volume_rotation\n    df[\'[\'supply_expansion_multiple\'] = supply_expansion_multiple\n    df[\'circulating_versus_max\'] = circulating_versus_max\n    df[\'market_cap_dilution_ratio\'] = market_cap_dilution_ratio - 1\n    except ZeroDivisionError:\n      print(\'null\')\n      \n\n    \n    #df_market = pd.DataFrame(data=mkdata, columns = [\'market_cap\',\'current_price\',\'fdv_to_tvl_ratio\',\'fully_diluted_valuation\',\'mcap_to_tvl_ratio\'])\n    \n    print(df_market)\n\n\n    \n    print(dict_return)\n\n\n\n    category = json_parsed[\'categories\']\n\n    print(\'\')  \n    print(\'CATEGORIES\')  \n    print(category)\n    \n    \n    \n    community_data = json_parsed[\'community_data\']\n\n    df_community_data = pd.DataFrame(data=community_data, index=[0])\n   

  0%|          | 0/3 [00:00<?, ?it/s]

'    \n    df[\'circulating_versus_max\'] = circulating_supply/total_supply\n    df[\'market_cap_dilution_ratio\'] =  fully_diluted_valuation / market_cap\n      \n          \n    df[\'dollar_volume_rotation\'] = dollar_volume_rotation\n    df[\'[\'supply_expansion_multiple\'] = supply_expansion_multiple\n    df[\'circulating_versus_max\'] = circulating_versus_max\n    df[\'market_cap_dilution_ratio\'] = market_cap_dilution_ratio - 1\n    except ZeroDivisionError:\n      print(\'null\')\n      \n\n    \n    #df_market = pd.DataFrame(data=mkdata, columns = [\'market_cap\',\'current_price\',\'fdv_to_tvl_ratio\',\'fully_diluted_valuation\',\'mcap_to_tvl_ratio\'])\n    \n    print(df_market)\n\n\n    \n    print(dict_return)\n\n\n\n    category = json_parsed[\'categories\']\n\n    print(\'\')  \n    print(\'CATEGORIES\')  \n    print(category)\n    \n    \n    \n    community_data = json_parsed[\'community_data\']\n\n    df_community_data = pd.DataFrame(data=community_data, index=[0])\n   

In [ ]:
#@title Load Token ID to SQLite { display-mode: "code" }
 
proprietary_dashboard = "proprietary_dashboard"

single.to_sql(proprietary_dashboard, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

proprietary_dashboard = pd.read_sql('select * from {}'.format(proprietary_dashboard), conn_SQLite)

proprietary_dashboard.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   market_cap                    0 non-null      object
 1   circulating_supply            0 non-null      object
 2   current_price                 0 non-null      object
 3   fdv_to_tvl_ratio              0 non-null      object
 4   price_change_24h              0 non-null      object
 5   mcap_to_tvl_ratio             0 non-null      object
 6   total_supply                  0 non-null      object
 7   total_volume                  0 non-null      object
 8   dollar_volume_rotation        0 non-null      object
 9   CoinGecko_id                  0 non-null      object
 10  price_change_percentage_7d    0 non-null      object
 11  price_change_percentage_14d   0 non-null      object
 12  price_change_percentage_30d   0 non-null      object
 13  price_change_percentage_60d   0 non-n

In [ ]:


path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/ohlc/watchlist/output/bulk/"


Cleaned_data_file = 'watchlist_Bulk_CoinGecko_ohlc.csv'

df = pd.DataFrame()

extension = 'csv'
#dataset_location = os.chdir(r'D:\investing_programs\datasets\CoinMetrics')
all_filenames = []
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/ohlc/watchlist/output/*.{}'.format(extension))]

#combine all files in the list
for i in all_filenames:
    df = pd.read_csv(i)#.read(),engine='openpyxl')   
    

combined_csv = df.append([pd.read_csv(f) for f in all_filenames])

del combined_csv['Unnamed: 0']

combined_csv.to_csv( path + Cleaned_data_file, index=False, encoding='utf-8-sig')


print(combined_csv.head())




            Date   Open   High    Low  Close CoinGecko_id    prop_timestamp
0  1620345600000  30.05  30.05  30.05  30.05      adamant  1651608571499549
1  1620691200000  77.62  77.62  37.21  37.21      adamant  1651608571499549
2  1621036800000  35.87  49.95  33.55  37.57      adamant  1651608571499549
3  1621382400000  39.16  45.40  39.16  45.40      adamant  1651608571499549
4  1621728000000  32.23  32.23  24.31  24.31      adamant  1651608571499549


In [ ]:
#@title Load Token ID to SQLite { display-mode: "code" }

watchlist_CoinGecko_ohlc = "watchlist_CoinGecko_ohlc"

combined_csv.to_sql(watchlist_CoinGecko_ohlc, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

watchlist_CoinGecko_ohlc = pd.read_sql('select * from {}'.format(watchlist_CoinGecko_ohlc), conn_SQLite)

In [ ]:
watchlist_CoinGecko_ohlc.head()
watchlist_CoinGecko_ohlc['Datetime'] = pd.to_datetime(watchlist_CoinGecko_ohlc['Date'], unit='ms')
 
watchlist_CoinGecko_ohlc['Datetime']

0       2021-05-07
1       2021-05-11
2       2021-05-15
3       2021-05-19
4       2021-05-23
           ...    
60099   2022-04-19
60100   2022-04-23
60101   2022-04-27
60102   2022-04-30
60103   2022-05-03
Name: Datetime, Length: 60104, dtype: datetime64[ns]

0       2021-05-07
1       2021-05-11
2       2021-05-15
3       2021-05-19
4       2021-05-23
           ...    
60099   2022-04-19
60100   2022-04-23
60101   2022-04-27
60102   2022-04-30
60103   2022-05-03
Name: Datetime, Length: 60104, dtype: datetime64[ns]

In [ ]:
watchlist_CoinGecko_ohlc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60104 entries, 0 to 60103
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            60104 non-null  int64         
 1   Open            60104 non-null  float64       
 2   High            60104 non-null  float64       
 3   Low             60104 non-null  float64       
 4   Close           60104 non-null  float64       
 5   CoinGecko_id    60104 non-null  object        
 6   prop_timestamp  60104 non-null  int64         
 7   Datetime        60104 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 3.7+ MB


In [68]:
#@title ok { display-mode: "code" }



cg = CoinGeckoAPI()
CoinGecko_id = df_proprietary_airtable_shortlisted['CoinGecko_id'].unique()

days = ['max','30','1']
vs_currency = ['usd']

frames = []
b_March = []
df_desc = {}


frames_description = []


try:

  for i in tqdm(CoinGecko_id[:]):

      try:          
        b = cg.get_coin_by_id(id=i)
      except:
          pass
     

      # description  
      df_desc['market_cap'] = b['market_data']['market_cap']['usd']
      df_desc['categories'] = b['categories']
      df_desc['price_change_24h'] = b['market_data']['price_change_24h'] 
      df_desc['total_volume'] = b['market_data']['total_volume']['usd']
      df_desc['circulating_supply'] = b['market_data']['circulating_supply']
      df_desc['description'] = b['description']['en']
      df_desc['categories'] = b['categories']
      df_desc['fdv_to_tvl_ratio'] = b['market_data']['fdv_to_tvl_ratio']
      df_desc['mcap_to_tvl_ratio'] = b['market_data']['mcap_to_tvl_ratio']
      df_desc['total_supply'] = b['market_data']['total_supply']
      df_desc['coingecko_score'] = b['coingecko_score'] 
      df_desc['homepage'] = b['links']['homepage'] 
      df_desc['ath_change_percentage'] = b['market_data']['ath_change_percentage']['usd'] 
      

    #Convert data to list of dataframes
      full_table = pd.json_normalize(df_desc)
    #results = [full_table]
      frames_description.append(full_table)
      #print(full_table)
      print(full_table)

      for day in days:
    
        try:  
          a = cg.get_coin_ohlc_by_id(
            id=i,
            vs_currency='usd',
            days = day)

        except:
            pass
   
       
        # Create dataframe and Chart 
        df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])
        df['CoinGecko_id'] = str(i)
        df['Datetime'] = pd.to_datetime(df['Date'], unit='ms')

        trace1 = {
        'x': df.Datetime,
        'open': df.Open,
        'close': df.Close,
        'high': df.High,
        'low': df.Low,
        'type': 'candlestick',
        'name': str(i).upper(),
        'showlegend': True
        }

      
        # Calculate and define moving average of 30 periods
        avg_10 = df.Close.rolling(window=10, min_periods=1).mean()
        # Calculate and define moving average of 50 periods
        avg_20 = df.Close.rolling(window=20, min_periods=1).mean()
        # Calculate and define moving average of 30 periods
        avg_50 = df.Close.rolling(window=50, min_periods=1).mean()
        # Calculate and define moving average of 50 periods
        avg_100 = df.Close.rolling(window=100, min_periods=1).mean()


        trace2 = {
            'x': df.Datetime,
            'y': avg_10,
            'type': 'scatter',
            'mode': 'lines',
            'line': {
                'width': 1,
                'color': 'purple'
                    },
            'name': 'Moving Average of 10 periods'
        }

        trace3 = {
            'x': df.Datetime,
            'y': avg_20,
            'type': 'scatter',
            'mode': 'lines',
            'line': {
                'width': 1,
                'color': 'orange'
            },
            'name': 'Moving Average of 20 periods'
        }
        trace4 = {
            'x': df.Datetime,
            'y': avg_50,
            'type': 'scatter',
            'mode': 'lines',
            'line': {
                'width': 1,
                'color': 'green'
                    },
            'name': 'Moving Average of 50 periods'
        }

        trace5 = {
            'x': df.Datetime,
            'y': avg_100,
            'type': 'scatter',
            'mode': 'lines',
            'line': {
                'width': 1,
                'color': 'blue'
            },
            'name': 'Moving Average of 100 periods'
        }
        data = [trace1, trace2, trace3,trace4,trace5]
        # Config graph layout
        layout = go.Layout({
            'title': {
                'text': str(i).upper().replace("-"," ") + ' Moving Averages',
                'font': {
                    'size': 15
                }
            }
        })


        # In[38]:


        fig = go.Figure(data=data, layout=layout)
        fig.show()
        CoinGeckoAPI
        time.sleep(60/50)



except ConnectionError:
    time.sleep(60)

fig.write_html("Moving Averages.html")


  0%|          | 0/22 [00:00<?, ?it/s]

   market_cap                                         categories  \
0     2663958  [DaoMaker Ecosystem, Binance Smart Chain Ecosy...   

   price_change_24h  total_volume  circulating_supply  \
0         -0.000166        203314         158699930.0   

                                         description fdv_to_tvl_ratio  \
0  "ShopNEXT is a Shop-To-Earn Crypto Onboarding ...             None   

  mcap_to_tvl_ratio  total_supply  coingecko_score  \
0              None  1.000000e+09           15.342   

                     homepage  ath_change_percentage  
0  [https://shopnext.io/, , ]              -95.59708  


   market_cap                                         categories  \
0   212128950  [Derivatives, Perpetuals, Avalanche Ecosystem,...   

   price_change_24h  total_volume  circulating_supply  \
0          0.028557       1753707        7.596526e+06   

                                         description  fdv_to_tvl_ratio  \
0  GMX is a decentralized spot and perpetual exch...               1.1   

   mcap_to_tvl_ratio  total_supply  coingecko_score  \
0               0.63  7.596526e+06           32.548   

                       homepage  ath_change_percentage  
0  [https://gmx.financial/, , ]              -55.03432  


   market_cap          categories  price_change_24h  total_volume  \
0    71932170  [Governance, None]         -0.031364       9076593   

   circulating_supply                                        description  \
0          77220001.0  Loot is randomized adventurer gear generated a...   

  fdv_to_tvl_ratio mcap_to_tvl_ratio  total_supply  coingecko_score  \
0             None              None    77220001.0           24.251   

                             homepage  ath_change_percentage  
0  [https://www.lootproject.com/, , ]              -87.91929  


   market_cap                                         categories  \
0   615176859  [Solana Ecosystem, Music, Non-Fungible Tokens ...   

   price_change_24h  total_volume  circulating_supply  \
0         -0.007952      23058759         713381683.0   

                                         description fdv_to_tvl_ratio  \
0  AUDIO is the native platform token of the Audi...             None   

  mcap_to_tvl_ratio  total_supply  coingecko_score                  homepage  \
0              None  1.091824e+09           33.467  [https://audius.co/, , ]   

   ath_change_percentage  
0              -82.55399  


...

   market_cap categories  price_change_24h  total_volume  circulating_supply  \
0     8058377     [None]         -0.027551        220081        1.022798e+08   

                                         description  fdv_to_tvl_ratio  \
0  StakeWise is a liquid Ethereum staking protoco...              0.37   

   mcap_to_tvl_ratio  total_supply  coingecko_score  \
0               0.04  1.000000e+09           13.597   

                      homepage  ath_change_percentage  
0  [https://stakewise.io/, , ]              -78.05714  


   market_cap categories  price_change_24h  total_volume  circulating_supply  \
0     8058377     [None]         -0.027551        220081        1.022798e+08   

                                         description  fdv_to_tvl_ratio  \
0  StakeWise is a liquid Ethereum staking protoco...              0.37   

   mcap_to_tvl_ratio  total_supply  coingecko_score  \
0               0.04  1.000000e+09           13.597   

                      homepage  ath_change_percentage  
0  [https://stakewise.io/, , ]              -78.05714  


   market_cap categories  price_change_24h  total_volume  circulating_supply  \
0     8058377     [None]         -0.027551        220081        1.022798e+08   

                                         description  fdv_to_tvl_ratio  \
0  StakeWise is a liquid Ethereum staking protoco...              0.37   

   mcap_to_tvl_ratio  total_supply  coingecko_score  \
0               0.04  1.000000e+09           13.597   

                      homepage  ath_change_percentage  
0  [https://stakewise.io/, , ]              -78.05714  


   market_cap                                         categories  \
0   113273809  [Lending/Borrowing, Yield Farming, Decentraliz...   

   price_change_24h  total_volume  circulating_supply  \
0         -0.003749       2678720        6.712055e+08   

                                         description  fdv_to_tvl_ratio  \
0  It is a DeFi uncollateralized lending protocol...              2.35   

   mcap_to_tvl_ratio  total_supply  coingecko_score  \
0                1.1  1.441129e+09           34.861   

                                            homepage  ath_change_percentage  
0  [http://truefi.io/, https://www.trusttoken.com...              -83.41524  


   market_cap                                  categories  price_change_24h  \
0    22169400  [Decentralized Exchange Token (DEX), None]           -0.0033   

   total_volume  circulating_supply  \
0       2927819         223450000.0   

                                         description  fdv_to_tvl_ratio  \
0  Based on ZK-Rollups technology, L2 Labs has la...              26.6   

   mcap_to_tvl_ratio  total_supply  coingecko_score                  homepage  \
0               5.94  1.000000e+09           21.299  [https://zks.org/en, , ]   

   ath_change_percentage  
0              -99.11718  


In [ ]:
df = pd.DataFrame()

b




#.categories

{'additional_notices': [],
 'asset_platform_id': 'binance-smart-chain',
 'block_time_in_minutes': 0,
 'categories': ['DaoMaker Ecosystem', 'Binance Smart Chain Ecosystem'],
 'coingecko_rank': 2311,
 'coingecko_score': 15.342,
 'community_data': {'facebook_likes': None,
  'reddit_accounts_active_48h': 0,
  'reddit_average_comments_48h': 0.0,
  'reddit_average_posts_48h': 0.0,
  'reddit_subscribers': 0,
  'telegram_channel_user_count': 3531,
  'twitter_followers': 57321},
 'community_score': 10.037,
 'contract_address': '0x9809e877192b510d767a94ba39a79429219a5afb',
 'country_origin': '',
 'description': {'ar': '"ShopNEXT is a Shop-To-Earn Crypto Onboarding Platform promoted by Binance. We onboard new-to-crypto users by rewarding them free crypto (Bitcoin, Ethereum, BNB, USDT) when shopping at ShopNEXT partners.\r\n\r\nIn addition, for every shopping transaction, ShopNEXT also rewards users extra NEXT, the cryptocurrency issued by ShopNEXT. Users can sell NEXT on the DEX and CEX exchanges

In [ ]:
PS: _desc



In [ ]:
#@title ok { display-mode: "code" }

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

colors = {
  'background': '#111111',
  'text': '#7FDBFF'
}

cg = CoinGeckoAPI()
CoinGecko_id = df_proprietary_airtable_watch_all['CoinGecko_id']
frames = []


for i in tqdm(CoinGecko_id[:1]):
  try:  
    a = cg.get_coin_ohlc_by_id(id=i, vs_currency='usd', days = 'max')
  except:
      pass
  
  # Create dataframe and Chart 
  df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])
  df['CoinGecko_id'] = str(i)

  trace1 = {
  'x': df.Date,
  'open': df.Open,
  'close': df.Close,
  'high': df.High,
  'low': df.Low,
  'type': 'candlestick',
  'name': str(i).upper(),
  'showlegend': True
}

  
  # Calculate and define moving average of 30 periods
  avg_10 = df.Close.rolling(window=10, min_periods=1).mean()
  # Calculate and define moving average of 50 periods
  avg_20 = df.Close.rolling(window=20, min_periods=1).mean()
  # Calculate and define moving average of 30 periods
  avg_50 = df.Close.rolling(window=50, min_periods=1).mean()
  # Calculate and define moving average of 50 periods
  avg_100 = df.Close.rolling(window=100, min_periods=1).mean()


  trace2 = {
      'x': df.Date,
      'y': avg_10,
      'type': 'scatter',
      'mode': 'lines',
      'line': {
          'width': 1,
          'color': 'yellow'
              },
      'name': 'Moving Average of 10 periods'
  }

  trace3 = {
      'x': df.Date,
      'y': avg_20,
      'type': 'scatter',
      'mode': 'lines',
      'line': {
          'width': 1,
          'color': 'orange'
      },
      'name': 'Moving Average of 20 periods'
  }
  trace4 = {
      'x': df.Date,
      'y': avg_50,
      'type': 'scatter',
      'mode': 'lines',
      'line': {
          'width': 1,
          'color': 'green'
              },
      'name': 'Moving Average of 50 periods'
  }

  trace5 = {
      'x': df.Date,
      'y': avg_100,
      'type': 'scatter',
      'mode': 'lines',
      'line': {
          'width': 1,
          'color': 'blue'
      },
      'name': 'Moving Average of 100 periods'
  }
  data = [trace1, trace2, trace3,trace4,trace5]
  # Config graph layout
  layout = go.Layout({
      'title': {
          'text': str(i).upper().replace("-"," ") + ' Moving Averages',
          'font': {
              'size': 15
          }
      }
  })


  # In[38]:


  fig = go.Figure(data=data, layout=layout)

  fig.show()
  time.sleep(60/50)

  app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
      html.H1(
          children='Hello Dash',
          style={
              'textAlign': 'center',
              'color': colors['text']
          }
      ),

      html.Div(children='Dash: A web application framework for your data.', style={
          'textAlign': 'center',
          'color': colors['text']
      }),

      dcc.Graph(
          id='example-graph-2',
          figure=fig
      )
  ])

  if __name__ == '__main__':
      app.run_server(debug=True)


In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
b.info()